# 0. Imports

In [1]:
!pip install parsimonious

from aloe.parser  import Parser
from aloe.program import AloeProgram

# 1. Parsing

In [2]:
with open("aloe/Examples/toparse.pl", 'r') as file:
    print(file.read())

set(verbose,0).

modeh(1,echec_cours(+personne)).
modeb(1,echec_pre(+personne)).

determination(echec_cours/1,echec_pre/1).
    
% BACKGROUND    
:- begin_bg.

personne(anne).
personne(marie).
personne(jean).
personne(jacques).

echec_pre(jean).
echec_pre(jacques).

    
:- end_bg.    

    
% POSITIVE EXAMPLES
:- begin_in_pos.

echec_cours(jean).
echec_cours(jacques).  

   
:- end_in_pos.

    

% NEGATIVE EXAMPLES
:- begin_in_neg.

echec_cours(anne).
echec_cours(marie).
    
:- end_in_neg.    



In [3]:
parser = Parser()
P = parser.parse("aloe/Examples/toparse.pl", 'aloefile')
print(P)

Background:
LogicProgram object
   personne(anne).
   personne(marie).
   personne(jean).
   personne(jacques).
   echec_pre(jean).
   echec_pre(jacques).

Examples:
Positive:
echec_cours(jean).
echec_cours(jacques).
Negative:
echec_cours(anne).
echec_cours(marie).

Modes:
modeh: {'echec_cours/1': modeh(1,echec_cours(+personne)).}
modeb: {'echec_pre/1': modeb(1,echec_pre(+personne)).}
determinations: {'echec_cours/1': OrderedSet(['echec_pre/1'])}

options:
verbose: 0
i: 2
c: 2
h: 10000
solver: AloeSolver
learner: ProgolLearner
hmetric: FnMetric
update_knowledge: True
logging: False


# 2. Deducing

## Echec example

In [4]:
# P.set('verbose',1)
P.query("echec_pre(X).")

(True,
          0     1
 X  jacques  jean)

## Animal example

In [5]:
bckg = """
animal(X) :- mammal(X).
animal(X) :- fish(X).

fish(truite).
fish(saumon).

mammal(rhino).
mammal(human).

mortel(X) :- animal(X).
"""

ap = AloeProgram.build_from_background(bckg)
ap.set('verbose',1)
ap.query('mortel(X).')


h 0
Atom mortel(X)
Candidates {mortel(X):-animal(X).}
Match {mortel(X):-animal(X).}
Clause mortel(X1):-animal(X1).
Substitution {X1: X}
Atoms [animal(X)]

h 1
Atom animal(X)
Candidates {animal(X):-mammal(X)., animal(X):-fish(X).}
Match {animal(X):-mammal(X)., animal(X):-fish(X).}
Clause animal(X2):-mammal(X2).
Substitution {X1: X, X2: X}
Atoms [mammal(X)]

h 2
Atom mammal(X)
Candidates {mammal(rhino)., mammal(human).}
Match {mammal(rhino)., mammal(human).}
Clause mammal(rhino).
Substitution {X1: rhino, X2: rhino, X: rhino}
Atoms []

h 3
Atom mammal(X)
Clause mammal(human).
Substitution {X1: human, X2: human, X: human}
Atoms []

h 4
Atom animal(X)
Clause animal(X2):-fish(X2).
Substitution {X1: X, X2: X}
Atoms [fish(X)]

h 5
Atom fish(X)
Candidates {fish(saumon)., fish(truite).}
Match {fish(saumon)., fish(truite).}
Clause fish(saumon).
Substitution {X1: saumon, X2: saumon, X: saumon}
Atoms []

h 6
Atom fish(X)
Clause fish(truite).
Substitution {X1: truite, X2: truite, X: truite}
Atoms [

(True,
        0      1       2       3
 X  rhino  human  saumon  truite)

## Substract example

In [6]:
bckg = """

dec(3,2).
dec(2,1).
dec(1,0).

sub(X,0,X).
sub(X,Y,Z):- dec(X,A), dec(Y,B), sub(A,B,Z).

"""

ap = AloeProgram.build_from_background(bckg)
ap.set('verbose',1)
ap.set('h',100)
ap.query('sub(X, Y, 2).')



h 0
Atom sub(X,Y,2)
Candidates {sub(X,0,X)., sub(X,Y,Z):-dec(X,A),dec(Y,B),sub(A,B,Z).}
Match {sub(X,0,X)., sub(X,Y,Z):-dec(X,A),dec(Y,B),sub(A,B,Z).}
Clause sub(X1,0,X1).
Substitution {X1: 2, Y: 0, X: 2}
Atoms []

h 1
Atom sub(X,Y,2)
Clause sub(X1,Y1,Z):-dec(X1,A),dec(Y1,B),sub(A,B,Z).
Substitution {X1: X, Y1: Y, Z: 2}
Atoms [sub(A,B,2), dec(Y,B), dec(X,A)]

h 2
Atom dec(X,A)
Candidates {dec(3,2)., dec(2,1)., dec(1,0).}
Match {dec(3,2)., dec(2,1)., dec(1,0).}
Clause dec(3,2).
Substitution {X1: 3, Y1: Y, Z: 2, X: 3, A: 2}
Atoms [sub(A,B,2), dec(Y,B)]

h 3
Atom dec(Y,B)
Candidates {dec(3,2)., dec(2,1)., dec(1,0).}
Match {dec(3,2)., dec(2,1)., dec(1,0).}
Clause dec(3,2).
Substitution {X1: 3, Y1: 3, Z: 2, X: 3, A: 2, Y: 3, B: 2}
Atoms [sub(A,B,2)]

h 4
Atom sub(2,2,2)
Candidates {sub(X,Y,Z):-dec(X,A),dec(Y,B),sub(A,B,Z).}
Match {sub(X,Y,Z):-dec(X,A),dec(Y,B),sub(A,B,Z).}
Clause sub(X2,Y2,Z1):-dec(X2,A1),dec(Y2,B1),sub(A1,B1,Z1).
Substitution {X1: 3, Y1: 3, Z: 2, X: 3, A: 2, Y: 3, B: 2, X

(True,
    0  1  2  3  4  5  6
 Y  0  3  3  3  2  2  1
 X  2  3  2  1  2  1  1)